In [ ]:
#importing libraries
import os
import sys
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

In [ ]:
#BASE_DIR = r'D:\Work\DS_and_AI\Data_Sets'
#BASE_DIR = r'/content'
#GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
glove_file = r'/content/glove.6B.100d.txt'
#print(GLOVE_DIR)
#MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


In [ ]:
#importing the dataset
!wget  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-11-12 12:52:44--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-11-12 12:52:44--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6aa86999d7cf08209ce0daa59c.dl.dropboxusercontent.com/zip_download_get/AmerJaNzsce66HU1Q3yivjj9UVRYFE5uEYick42dix8QMUJzwbpASj7d4-t5-iqEyrkcLzMVkaicatlpiVm1qbw058CQ_cqNWYOYTv_1ef7cYg [following]
--2020-11-12 12:52:44--  https://uc6aa86999d7cf08209ce0daa59c.dl.dropboxusercontent.com/zip_download_get/AmerJaNzsce66HU1Q3yivjj9UVRYFE5uEYick42dix8QMUJzwbpASj7d4-t5-iqEyrkcLzMVkaicatlpiVm1

In [ ]:
#unzipping the dataset
!unzip '/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0'

Archive:  /content/AAAw1_JIzpuVvwteJCma0xMla?dl=0
mapname:  conversion of  failed
 extracting: train.csv               

1 archive had fatal errors.


In [ ]:
#importing glove.6B
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-11-12 12:52:59--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-12 12:52:59--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-12 12:52:59--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
#unzipping the glove files
!unzip '/content/glove.6B.zip'

Archive:  /content/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
#reading the dataset
dataset = r'/content/train.csv'

In [ ]:
data = pd.read_csv(dataset)

In [ ]:
data

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [ ]:
#dropping qid column
data.drop(['qid'],axis=1,inplace=True)

In [ ]:
data

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0
...,...,...
1306117,What other technical skills do you need as a c...,0
1306118,Does MS in ECE have good job prospects in USA ...,0
1306119,Is foam insulation toxic?,0
1306120,How can one start a research project based on ...,0


In [ ]:
#format our text samples and labels into tensors that can be fed into a neural network
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['question_text'])
sequences = tokenizer.texts_to_sequences(data['question_text'])

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 222161 unique tokens.


In [ ]:
#finding the max_elnth of the sequence
max_len = 0
i= 0 
for i in range(len(sequences)):
    if len(sequences[i]) > max_len:
        max_len = len(sequences[i])
        print(max_len)
        print(sequences[i])
print(max_len)
        
    

13
[9, 48, 6683, 7219, 158, 55, 6107, 36, 4, 1206, 6, 1, 8333]
16
[11, 14, 24, 29, 3864, 498, 9, 35, 14, 3672, 37, 5, 3089, 10, 44, 1846]
18
[16, 26, 104, 2559, 1642, 1455, 2440, 32, 4295, 64, 26, 44, 11, 1, 142, 13, 1165, 897]
45
[3, 17, 96, 54, 23, 24, 14, 89, 114, 6, 63, 2182, 15421, 14, 1375, 3148, 5, 1, 37, 14, 654, 1712, 901, 18808, 55, 1088, 1045, 52, 14, 33, 5, 24, 194, 106, 28, 79, 10, 98, 6911, 43, 6580, 9, 48, 146, 166]
47
[16, 20, 4, 227, 40, 58, 125, 54, 136, 185, 434, 40, 51, 310, 81, 12, 1547, 5, 10, 42, 286, 73, 305, 16, 8, 1142, 32, 77, 2525, 54, 136, 3, 77, 501, 13, 7560, 32, 39, 8, 459, 13, 157, 231, 631, 43, 4, 70]
48
[16, 11, 37, 618, 42, 175, 170, 3, 63, 2428, 187, 10, 2942, 554, 31, 85, 5, 24, 17, 8, 100, 445, 20, 37, 1057, 170, 20, 1379, 660, 64, 8, 303, 32, 47, 618, 42, 18, 29341, 1082, 118, 1371, 2470, 78, 22, 74, 23, 83, 6948]
50
[362, 18, 238, 1142, 82, 5, 54, 43, 4118, 10, 488, 30567, 1528, 8, 669, 18, 605, 10, 8, 24, 99529, 8, 96, 1211, 99530, 8, 2995, 995

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X_data = pad_sequences(sequences, maxlen=max_len)


Found 222161 unique tokens.


In [ ]:

labels = to_categorical(np.asarray(data['target']))
#labels = pd.DataFrame(labels)
#labels[0:24]

print(labels.shape)
print(X_data.shape)

(1306122, 2)
(1306122, 190)


In [ ]:
# split the data into a training set and a validation set
indices = np.arange(X_data.shape[0])
np.random.shuffle(indices)
X_data = X_data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_data.shape[0])
print(nb_validation_samples)

x_train = X_data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X_data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

261224


In [ ]:
x_train

array([[    0,     0,     0, ...,     7,    73,  3415],
       [    0,     0,     0, ...,    23,  2530, 17768],
       [    0,     0,     0, ...,  3401,    22,  2553],
       ...,
       [    0,     0,     0, ...,   431,     6,   198],
       [    0,     0,     0, ...,   879, 19059,   600],
       [    0,     0,     0, ...,  2319,    43,    66]], dtype=int32)

In [ ]:
y_train

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
#preparing the embedding layer
embeddings_index = {}
#f = os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')
f = glove_file
f = open(f,encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
#At this point we can leverage our embedding_index dictionary and our word_index to compute our embedding matrix:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(222162, 100)

In [ ]:
#We load this embedding matrix into an Embedding layer. Note that we set trainable=False to prevent the weights from being updated during training.
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [ ]:
#Finally we can then build a small 1D convnet to solve our classification problem:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
#x = Conv1D(128, 5, activation='relu')(x)
#x = MaxPooling1D(5)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(2, activation='sigmoid')(x)

model = Model(sequence_input, outputs)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
#fitting
model.fit(x_train, y_train, validation_data=(x_val,y_val),
          epochs=5, batch_size=128)

Epoch 1/5
8164/8164 [==============================] - 63s 8ms/step - loss: 0.2023 - acc: 0.9427 - val_loss: 0.1957 - val_acc: 0.9441
Epoch 2/5
8164/8164 [==============================] - 64s 8ms/step - loss: 0.1978 - acc: 0.9437 - val_loss: 0.1956 - val_acc: 0.9443
Epoch 3/5
8164/8164 [==============================] - 64s 8ms/step - loss: 0.1958 - acc: 0.9444 - val_loss: 0.1956 - val_acc: 0.9445
Epoch 4/5
8164/8164 [==============================] - 64s 8ms/step - loss: 0.1937 - acc: 0.9453 - val_loss: 0.1966 - val_acc: 0.9442
Epoch 5/5
8164/8164 [==============================] - 64s 8ms/step - loss: 0.1915 - acc: 0.9462 - val_loss: 0.2010 - val_acc: 0.9432


In [ ]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
#x = Conv1D(128, 5, activation='relu')(x)
#x = MaxPooling1D(5)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(2, activation='sigmoid')(x)

model = Model(sequence_input, outputs)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
#fitting
model.fit(x_train, y_train, validation_data=(x_val,y_val),
          epochs=5, batch_size=128)

Epoch 1/5
8164/8164 [==============================] - 64s 8ms/step - loss: 0.2023 - acc: 0.9425 - val_loss: 0.1984 - val_acc: 0.9424
Epoch 2/5
8164/8164 [==============================] - 64s 8ms/step - loss: 0.1979 - acc: 0.9438 - val_loss: 0.1957 - val_acc: 0.9446
Epoch 3/5
8164/8164 [==============================] - 64s 8ms/step - loss: 0.1957 - acc: 0.9445 - val_loss: 0.1953 - val_acc: 0.9445
Epoch 4/5
8164/8164 [==============================] - 64s 8ms/step - loss: 0.1936 - acc: 0.9453 - val_loss: 0.1972 - val_acc: 0.9443
Epoch 5/5
8164/8164 [==============================] - 64s 8ms/step - loss: 0.1915 - acc: 0.9462 - val_loss: 0.1991 - val_acc: 0.9443


In [ ]:
#prediction on x_val
y_pred = model.predict(x_val)


In [ ]:
y_pred [y_pred > 0.5] = 1.0
y_pred[y_pred <= 0.5] = 0.0

In [ ]:
y_pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
from  sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_val,y_pred)

In [ ]:
y_val

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
auc

0.5744431937164001

In [ ]:
pred = pd.DataFrame(y_pred)


In [ ]:
#exporting predictions to predictions file
pred.to_csv('predictions.csv',index=False)

In [ ]:
#saving weights
model.save_weights('Dangle_Priya_P3.h5')